# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,14.34,95,100,3.13,NZ,1714503877
1,1,guerrero negro,27.9769,-114.0611,19.90,54,92,4.21,MX,1714503877
2,2,vilyuchinsk,52.9306,158.4028,2.15,75,3,5.11,RU,1714503877
3,3,malangali,-8.5667,34.8500,17.45,86,32,5.02,TZ,1714503877
4,4,edinburgh of the seven seas,-37.0676,-12.3116,13.40,72,100,13.36,SH,1714503878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"]  <= 26.667) & (city_data_df["Max Temp"] >= 21.111) \
                                & (city_data_df["Wind Speed"]< 2.235)\
                                & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,turpan,42.9333,89.1667,21.44,9,0,1.71,CN,1714503894
109,109,pisco,-13.7000,-76.2167,26.03,57,0,1.54,PE,1714503896
223,223,colonia,50.9333,6.9500,21.64,60,0,2.06,DE,1714503917
286,286,gebeit,21.0667,36.3167,25.14,56,0,1.63,SD,1714503927
481,481,mbekenyera,-10.0000,38.9833,21.57,88,0,1.93,TZ,1714503962


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,turpan,CN,42.9333,89.1667,9,
109,pisco,PE,-13.7000,-76.2167,57,
223,colonia,DE,50.9333,6.9500,60,
286,gebeit,SD,21.0667,36.3167,56,
481,mbekenyera,TZ,-10.0000,38.9833,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
turpan - nearest hotel: Oriental
pisco - nearest hotel: La Portada
colonia - nearest hotel: Wasserturm Hotel Cologne
gebeit - nearest hotel: No hotel found
mbekenyera - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
95,turpan,CN,42.9333,89.1667,9,Oriental
109,pisco,PE,-13.7000,-76.2167,57,La Portada
223,colonia,DE,50.9333,6.9500,60,Wasserturm Hotel Cologne
286,gebeit,SD,21.0667,36.3167,56,No hotel found
481,mbekenyera,TZ,-10.0000,38.9833,88,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)